In [1]:
# Import required libraries
import pandas as pd
import numpy as np
import os

# Visualization (for insights)
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
!pip install db-dtypes

In [3]:
import db_dtypes
from google.cloud import bigquery

# Set up BigQuery client
client = bigquery.Client()

# Define query
query = """
SELECT * FROM `scalable-streaming-analytics.streaming_data.events`
LIMIT 1000
"""

# Load data into Pandas DataFrame
df = client.query(query).to_dataframe()

# Display basic info
df.info()
df.head()

C:\Users\bthea\scalable-streaming-analytics\venv\Lib\site-packages\google\cloud\bigquery\table.py:1820: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   user_id     1000 non-null   Int64  
 1   event       1000 non-null   object 
 2   content_id  1000 non-null   object 
 3   timestamp   1000 non-null   float64
dtypes: Int64(1), float64(1), object(2)
memory usage: 32.4+ KB


,user_id,event,content_id,timestamp
0,1,watch,content_5,1.739140e+09
1,1,like,content_34,1.739506e+09
2,1,like,content_37,1.739599e+09
3,1,like,content_6,1.739600e+09
4,1,like,content_44,1.739600e+09


In [4]:
# Check missing values
print("Missing Values:\n", df.isnull().sum())

# Check data types
print("\nData Types:\n", df.dtypes)

Missing Values:
 user_id       0
event         0
content_id    0
timestamp     0
dtype: int64

Data Types:
 user_id         Int64
event          object
content_id     object
timestamp     float64
dtype: object


In [5]:
# Convert timestamp column to datetime
df['timestamp'] = pd.to_datetime(df['timestamp'], unit='s')

# Encode categorical values if necessary
df['event'] = df['event'].astype("category")

In [8]:
# Remove duplicates
df.drop_duplicates(inplace=True)

# Identify outliers using IQR
Q1 = df['timestamp'].quantile(0.25)
Q3 = df['timestamp'].quantile(0.75)
IQR = Q3 - Q1

# Define outlier range
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# Filter outliers
df = df[(df['timestamp'] >= lower_bound) & (df['timestamp'] <= upper_bound)]
print(f"IQR: {IQR}")

IQR: 0 days 00:30:48.690471168


C:\Users\bthea\AppData\Local\Temp\ipykernel_25720\2640786360.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop_duplicates(inplace=True)


In [9]:
df['timestamp'] = pd.to_datetime(df['timestamp'], unit='s')
print(df.head())

    user_id event  content_id                     timestamp
5         1  like   content_9 2025-02-14 20:13:27.839601994
6         1  like  content_22 2025-02-14 20:16:39.028926134
7         1  like  content_32 2025-02-14 20:39:34.522868395
8         1  like  content_48 2025-02-14 20:59:11.585451126
13        2  like  content_50 2025-02-14 20:27:24.862315178


In [10]:
df.drop_duplicates(inplace=True)

In [11]:
print(df.isnull().sum())  # Check missing values
df.dropna(inplace=True)  # Drop if needed

user_id       0
event         0
content_id    0
timestamp     0
dtype: int64


In [12]:
df = df.sort_values(by='timestamp')